In [ ]:
import numpy as np
from astropy import units as u
from astropy import time

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.twobody.propagation import propagate
from poliastro.examples import iss

from poliastro.twobody.propagation import cowell
from poliastro.plotting import OrbitPlotter3D
from poliastro.util import norm

from perylune import orbit_tools

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
# Perturber 1: This is a constant forward acceleration. This could model an ion engine.
#
# Based on poliastro example.
accel = 2e-5

def pert_constant_accel(accel):
    def constant_acc(t0, u, k):
        """ My **guesses** at the parameters:
        t0 - time in seconds since beginning
        u - state vectors, u[0:2] is r (position), u[3:5] - velocity
        k - GM constant, expressed in km3/s2 (so it's 398600.4418 for Earth)
        return 3 values - x,y,z acceleration
        """
        
        v = u[3:]
        norm_v = (v[0]**2 + v[1]**2 + v[2]**2)**.5 # expressed in km/s
        print(norm_v)
        return accel * v / norm_v
        #return 0,0,0.0001

    return constant_acc

In [ ]:
# Perturber 2: Constant brake. This slowly slows down (constant deceleration)
accel2 = -2e-5


In [ ]:
# Perturber 3: Solar pressure. Uses solar pressure of the Sun. Parameters:

solar_rad_pressure = 4.5e-6 # Pascals, or N/m2, source: https://en.wikipedia.org/wiki/Radiation_pressure#Pressures_of_absorption_and_reflection

area = 4 # sail area in m2, source: https://pw-sat.pl/wp-content/uploads/2018/11/PW-Sat2-w-liczbach-1.pdf
mass = 2.5 # mass of the sat in kilograms, source: https://pw-sat.pl/wp-content/uploads/2018/11/PW-Sat2-w-liczbach-1.pdf

f = solar_rad_pressure * area

a = solar_rad_pressure * area / mass

print(f,a) # m/s^2


def pert_solar_sail(accel):

    RE = 6378.137 # Earth radius
    def in_shadow(v):
        """ Determines whether an object is in shadow or sunlit. It assumes the Sun is at [147000000,0,0] position. """

        if v[0] > 0: 
            return True # This spacecraft is above terminator, so it's always lit.
        r = (v[1]**2 + v[2]**2)**.5

        return r > RE # If the spacecraft is farther away from the X axis than the Earth radius, it is above the Earth shadow.

    def accel_solar(t0, u, k):
        """ My **guesses** at the parameters:
        t0 - time in seconds since beginning
        u - state vectors, u[0:2] is r (position), u[3:5] - velocity
        k - GM constant, expressed in km3/s2 (so it's 398600.4418 for Earth)
        return 3 values - x,y,z acceleration
        """
        
        r = u[:3]
        v = u[3:]
        norm_v = (v[0]**2 + v[1]**2 + v[2]**2)**.5 # expressed in km/s
        print(in_shadow(r))
        #print(norm_v)
        #return accel * v / norm_v
        return 0,0,0.000001

    return accel_solar

In [ ]:
times = np.linspace(0, 2 * iss.period, 50)
orbit_tools.print_orb(iss)

In [ ]:
positions = propagate(
    iss,
    time.TimeDelta(times),
    method=cowell,
    rtol=1e-11,
    #ad=pert_constant_accel(accel),
    ad=pert_solar_sail(accel)
)

In [ ]:
frame = OrbitPlotter3D()

frame.set_attractor(Earth)
frame.plot_trajectory(positions, color="green", label="ISS")

# Let's pretend this is the Sun
frame._draw_point(696*u.km, "red", "Sun", center=[14700, 0, 0] * u.km)


#frame._draw_point(696*u.km, "red", "X", center=[14700, 0, 0] * u.km)
#frame._draw_point(696*u.km, "yellow", "Y", center=[0, 14700, 0] * u.km)
#frame._draw_point(696*u.km, "green", "Z", center=[0, 0, 14700] * u.km)
frame._layout.width = 1200
frame._layout.height = 800
frame._layout.margin=dict(l=10, r=10, b=10, t=10, pad=4 )
frame.show()

In [ ]:
Earth.k

In [ ]:
iss.period*10